In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob
import collections
from collections import Counter
from scipy.special import expit
import string

In [2]:
!gdown --id 1n0j5zbMXMVnLPUqadFNcCxiZRu0ChoXL
!unzip -q "Programming_Assignment_3.zip" -d ""

Downloading...
From: https://drive.google.com/uc?id=1n0j5zbMXMVnLPUqadFNcCxiZRu0ChoXL
To: /content/Programming_Assignment_3.zip
42.0MB [00:00, 74.2MB/s]


In [3]:
test = pd.DataFrame()
train = pd.DataFrame()
label_list = []
content = []

In [4]:
def file_reading(datatype,directory):
#   files = glob.glob('Dataset/'+datatype+'/'+directory+'/*.txt',  recursive = True)
    files = glob.glob('Dataset/'+datatype+'/'+directory+'/*.txt',  recursive = True)

    print("Files are :",len(files))
  # return
    if directory == 'neg':
        label = 0
    else:
        label = 1

    print("Sentiment is: ",label)
    for file in files:
        f = open(file,'r')
        content.append(f.read())
        label_list.append(label)
    # print('Content:  ',content[0])


In [5]:
file_reading('test','neg')
file_reading('test','pos')

test['Review'] = content
test['Label'] = label_list

Files are : 12500
Sentiment is:  0
Files are : 12500
Sentiment is:  1


In [6]:
label_list = []
content = []
file_reading('train','neg')
file_reading('train','pos')
train['Review'] = content
train['Label'] = label_list

Files are : 12500
Sentiment is:  0
Files are : 12500
Sentiment is:  1


In [7]:
def preprocessing(data):
  f = open('Dataset/stop_words.txt','r')
  stop_words = f.read()
  punctuation = string.punctuation

  data['Review'] = data['Review'].str.replace('<br />',' ')
  data['Review'] = data['Review'].str.replace('[^\w\s*-]',' ')
  data['Review'] = data['Review'].str.replace('[\d]',' ')
  data['Review'] = data['Review'].str.lower()
  data['without_stopwords'] = data['Review'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))
  data['Review'] = data['without_stopwords']
  data.drop(columns='without_stopwords',inplace = True)
  
  return data

In [8]:
train = preprocessing(train)
test = preprocessing(test)

In [9]:
train.head()

,Review,Label
0,superdome one movies makes wonder made whole p...,0
1,saw remember advertisements movie interested f...,0
2,okay say revealing monster saying really fit c...,0
3,possible spoiler think story involving archie ...,0
4,like movie genre good gangster movies bad gang...,0


# Part 1

In [10]:
type(train['Label'][0])


numpy.int64

In [11]:
def unique_words(D):
  unique_list = set()
  list_words = D.str.split()
  list_words.apply(unique_list.update)
  unique_list = list(unique_list)

  return unique_list, len(unique_list)

In [12]:
def vocab(D):
  count = 0
  total_vocab = []
  list_words = D.str.split()
  for i in list_words:
    count = count + len(i)
    for w in i:
      total_vocab.append(w)
  # print(list_words)
  return count,total_vocab

In [13]:
# cnt, list_v = vocab(train['Review'])
# list_v[:5]
len(train[train['Label']==0]['Review'])
uniq_ls,cnt = (unique_words(train['Review']))
len(uniq_ls)

93647

In [14]:
# unique_words(train['Review'])
# cnt

In [15]:
# unique_list = set()
# list_words = train['Review'].str.split()
# list_words.apply(unique_list.update)
# unique_list = list(unique_list)
# len(unique_list)

In [16]:
def alt_denomenator(V,list_of_words):
  print('Calculating denomenator for likelihood! (using alt method)')
  total = len(list_of_words)
  unique_words = len(V)

  count = total+unique_words
  return count

In [17]:
def denominator(V,list_of_words):
  print('Calculating denomenator for likelihood!')
  print(list_of_words[0:5])
  print('number of words in V: ',len(V))
  count = 0
  for word in V:
    word_cnt = list_of_words.count(word)
    count = count + (word_cnt + 1)
  return count

In [18]:
def train_NB(D,C):
  print(D['Review'].head())
  count = {}
  loglikelihood = {}
  logprior = np.zeros(len(C))
  bigdoc = {}
  for class_i in C:
    print("Running for class ",class_i)
    N_doc = len(D['Review'])
    N_class = len(D[D['Label']==class_i]['Review'])
    logprior[int(class_i)] = np.log(N_class / N_doc)
    V , cntV= unique_words(D['Review'])
    _, bigdoc[int(class_i)] = vocab(D[D['Label']==class_i]['Review'])
    # sampled_denom = denominator(V,bigdoc[class_i])
    sampled_denom2 = alt_denomenator(V,bigdoc[class_i])
    # print('Value of denom ',sampled_denom)
    print('Value of denom 2 ',sampled_denom2)
    # return 0
    print('Calculating Log liklihood Next')
    word_cnt = Counter(bigdoc[class_i])
    for word in V:
      if word in word_cnt.keys():
        count[(word,class_i)] = word_cnt[word]
      else:
        count[(word,class_i)] = 0
      # sampled_denom = alt_denomenator(V,bigdoc[class_i])
      # print('Value of denom ',sampled_denom)
      # count[(word,class_i)] = bigdoc[class_i].count(word)
      # if count == 0:
      #   print(word)
      loglikelihood[(word,class_i)] = np.log((count[(word,class_i)] + 1) / sampled_denom2)
      # print(loglikelihood)
  
  return logprior,loglikelihood,V



In [19]:
C = list(set(train['Label']))
C

[0, 1]

In [20]:
# train_NB(train,C)

In [21]:
logprior,loglikelihood,V = train_NB(train,C)

0    superdome one movies makes wonder made whole p...
1    saw remember advertisements movie interested f...
2    okay say revealing monster saying really fit c...
3    possible spoiler think story involving archie ...
4    like movie genre good gangster movies bad gang...
Name: Review, dtype: object
Running for class  0
Calculating denomenator for likelihood! (using alt method)
Value of denom 2  1520311
Calculating Log liklihood Next
Running for class  1
Calculating denomenator for likelihood! (using alt method)
Value of denom 2  1586003
Calculating Log liklihood Next


In [22]:
# loglikelihood[('resignation',0)]

In [23]:
# np.log(0.5)

In [24]:
def test_NB(testdoc,logprior,loglikelihood,C,V):
  sum = np.zeros(len(C))
  set_V = set(V)
  # print(len(V),len(set_V))
  # return 0
  for class_i in C:
    sum[class_i] = logprior[class_i]
    for i in testdoc.split():
      word = i
      # print(word)
      # return 0
      if word in set_V:
        sum[class_i] = sum[class_i]+ loglikelihood[(word,class_i)]
  return np.argmax(sum)


In [25]:
# lbl = test_NB(test['Review'][i],logprior,loglikelihood,C,V)
# lbl

In [26]:
predicted_lbls = []
for i in range(len(test['Review'])):
  lbl = test_NB(test['Review'][i],logprior,loglikelihood,C,V)
  predicted_lbls.append(lbl)
  # if i % 1000 ==0:
  #   print('Running ', i)  
  # if i == 100:
  #   # print('Running')
  #   break




In [27]:
test['Predicted'] = predicted_lbls
test.head()

,Review,Label,Predicted
0,word honor erased vocabularies nations aggrava...,0,0
1,starts fairly well quite disturbing quickly si...,0,0
2,pretentious - varying degrees - watchable coll...,0,0
3,american movie industry tries critically look ...,0,0
4,much love film fan bad films love film bad goo...,0,0


In [28]:
def classification_accuracy(actual, predicted):
    total = len(actual)
    correct = 0
    for i in range(total):
        if actual[i] == predicted[i]:
            correct = correct + 1
  # query = np.where(actual == predicted,1,0)
  # occurrence_counts = Counter(query)
  # correct = occurrence_counts[1]
    accuracy = (correct/total) * 100

    return accuracy

In [29]:
print('Accuracy of the model is: ',classification_accuracy(test['Label'],test['Predicted']),'%')


Accuracy of the model is:  82.852 %


# Part 2

In [30]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

In [31]:
vectorizer = CountVectorizer()
train_X = vectorizer.fit_transform(train['Review'])
train_y = train['Label']

test_X = vectorizer.transform(test['Review'])
test_y = train['Label']

In [32]:
clf = MultinomialNB()
clf.fit(train_X, train_y)

y_predicted = clf.predict(test_X)
print("The accuracy is : ",accuracy_score(test_y, y_predicted))
print('Confusion Matrix:\n',confusion_matrix(test_y, y_predicted))

The accuracy is :  0.82564
Confusion Matrix:
 [[11015  1485]
 [ 2874  9626]]
